In [1]:
import openai
import pandas as pd
import pyodbc

import os
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
# drivers = pyodbc.drivers()
# print(drivers)

In [3]:
openai.api_key = os.getenv("OPENAI_API_KEY")

In [4]:
# Server=EAS-82NKM;Database=AdventureWorks2022;Trusted_Connection=True;

In [5]:
conn = pyodbc.connect(f'DRIVER={os.getenv("DRIVER_17")};'
                      f'SERVER={os.getenv("AW_SERVER_NAME")};'
                      f'DATABASE={os.getenv("AW_DATABASE_NAME")};'
                      'Trusted_Connection=Yes;')
print(f"Connection Live: {not(conn.closed)}")

Connection Live: True


In [6]:
cursor = conn.cursor()
cursor.execute("SELECT CONCAT(TABLE_SCHEMA, '.', TABLE_NAME) AS full_table_name FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_TYPE = 'BASE TABLE'")
tables = cursor.fetchall()

In [7]:
tables_names_list = [i[0] for i in tables]

In [8]:
tables_names_list

['Sales.SalesTaxRate',
 'Sales.PersonCreditCard',
 'Person.PersonPhone',
 'Sales.SalesTerritory',
 'Person.PhoneNumberType',
 'Production.Product',
 'Sales.SalesTerritoryHistory',
 'Production.ScrapReason',
 'HumanResources.Shift',
 'Production.ProductCategory',
 'Purchasing.ShipMethod',
 'Production.ProductCostHistory',
 'Production.ProductDescription',
 'Sales.ShoppingCartItem',
 'Production.ProductDocument',
 'dbo.DatabaseLog',
 'Production.ProductInventory',
 'Sales.SpecialOffer',
 'dbo.ErrorLog',
 'Production.ProductListPriceHistory',
 'Person.Address',
 'Sales.SpecialOfferProduct',
 'Production.ProductModel',
 'Person.AddressType',
 'Person.StateProvince',
 'Production.ProductModelIllustration',
 'dbo.AWBuildVersion',
 'Production.ProductModelProductDescriptionCulture',
 'Production.BillOfMaterials',
 'Sales.Store',
 'Production.ProductPhoto',
 'Production.ProductProductPhoto',
 'Production.TransactionHistory',
 'Production.ProductReview',
 'Person.BusinessEntity',
 'Production.T

In [9]:
# tables_needed = ["AIF_Jobs","SFDMON2"]
# table_names_list = [row[0] for row in tables if (row[0] == tables_needed[0] or row[0] == tables_needed[1])]

In [10]:
table_mapper = {}
for i,name in enumerate(tables_names_list):
    table_mapper.update({i:name})

In [11]:
table_mapper

{0: 'Sales.SalesTaxRate',
 1: 'Sales.PersonCreditCard',
 2: 'Person.PersonPhone',
 3: 'Sales.SalesTerritory',
 4: 'Person.PhoneNumberType',
 5: 'Production.Product',
 6: 'Sales.SalesTerritoryHistory',
 7: 'Production.ScrapReason',
 8: 'HumanResources.Shift',
 9: 'Production.ProductCategory',
 10: 'Purchasing.ShipMethod',
 11: 'Production.ProductCostHistory',
 12: 'Production.ProductDescription',
 13: 'Sales.ShoppingCartItem',
 14: 'Production.ProductDocument',
 15: 'dbo.DatabaseLog',
 16: 'Production.ProductInventory',
 17: 'Sales.SpecialOffer',
 18: 'dbo.ErrorLog',
 19: 'Production.ProductListPriceHistory',
 20: 'Person.Address',
 21: 'Sales.SpecialOfferProduct',
 22: 'Production.ProductModel',
 23: 'Person.AddressType',
 24: 'Person.StateProvince',
 25: 'Production.ProductModelIllustration',
 26: 'dbo.AWBuildVersion',
 27: 'Production.ProductModelProductDescriptionCulture',
 28: 'Production.BillOfMaterials',
 29: 'Sales.Store',
 30: 'Production.ProductPhoto',
 31: 'Production.Product

In [12]:
def choose_table(available_tables_list):
    while True:
        table_chosen = input(f"Select the number of the table to query from {available_tables_list}: ")
        try:
            name_selected = available_tables_list[int(table_chosen)]
            all_good = input(f"Are you sure you want to query from {name_selected}: Y/N")
            if (all_good == "Y" or all_good == "y"):
                return name_selected
            elif (all_good == "N" or all_good == "n"):
                print("Let's try again")
            else:
                print("Neither 'Y' or 'N' were selected, try again")
        except:
            print("Error in the selection, try again...")


In [13]:
chosen_table = choose_table(table_mapper)
partial_chosen_table = chosen_table.split(".")[1]

In [14]:
chosen_table, partial_chosen_table

('Sales.SalesPersonQuotaHistory', 'SalesPersonQuotaHistory')

In [15]:
def get_columns(cursor,table_name):
    columns = [column[3] for column in cursor.columns(table=table_name)]
    return columns

In [16]:
# columns = get_columns(cursor, partial_chosen_table)
# print(f"Columnas de la tabla {chosen_table}: {columns}")

In [17]:
def get_user_input():
    columns = get_columns(cursor, partial_chosen_table)
    print(f"The {chosen_table}'s columns are: {columns}")
    return input("Enter your query: ")

In [18]:
def complete_prompt():
    columns = get_columns(cursor, partial_chosen_table)
    user_query = get_user_input()
    return f"Knowing that the {chosen_table} Columns are: {columns}. Query to answer: {user_query}. Answer directly with 'SELECT', omit the ``` and the 'sql'."

In [19]:
complete_prompt = complete_prompt()
complete_prompt

The Sales.SalesPersonQuotaHistory's columns are: ['BusinessEntityID', 'QuotaDate', 'SalesQuota', 'rowguid', 'ModifiedDate']


"Knowing that the Sales.SalesPersonQuotaHistory Columns are: ['BusinessEntityID', 'QuotaDate', 'SalesQuota', 'rowguid', 'ModifiedDate']. Query to answer: dime cual fue la mejor cuota. Answer directly with 'SELECT', omit the ``` and the 'sql'."

In [20]:
response = openai.chat.completions.create(
    model = "gpt-4o",
    messages=[
        {"role": "system", "content": "You're an SQL query generating assistant."},
        {"role": "user", "content": complete_prompt}],
        temperature=0
)

In [21]:
def process_response(res):
    raw_query = res.choices[0].message.content.strip()
    if not raw_query.upper().startswith("SELECT"):
        # raw_query = f"SELECT {raw_query}"
        print(f"Unsupported query: {raw_query}. \nQuery must start with 'SELECT'")
        return ""
    elif raw_query.upper().startswith("SELECT"):
        return raw_query

In [22]:
print(process_response(response))

SELECT MAX(SalesQuota) AS MejorCuota FROM Sales.SalesPersonQuotaHistory


In [23]:
def execute_sql_query(response):
    final_query = process_response(response)
    cursor.execute(final_query)
    table = cursor.fetchall()
    return table

In [24]:
resulting_table = execute_sql_query(response)

In [25]:
type(resulting_table)

list

In [26]:
display(resulting_table)

[(Decimal('1898000.0000'),)]

In [27]:
display(pd.DataFrame([list(i) for i in resulting_table]))

,0
0,1898000.0000


In [28]:
cursor.close()
del cursor
conn.close()
print(f"Connection Closed: {conn.closed}")

Connection Closed: True


In [29]:
conn.closed

True

In [30]:
# conn.close()